In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
import random
InteractiveShell.ast_node_interactivity = "all"
import os
from datetime import datetime
import matplotlib.pyplot as plt
#import plotly.express as px
import glob
from tqdm import tqdm
from sklearn import preprocessing
import sweetviz as sv

In [3]:
# import data
url = '../data/australia_viirs.csv'
fileName = '../data/australia_viirs'
aus_fires = pd.read_csv(url, parse_dates=['acq_date'])
aus_fires.shape
aus_fires.head()
aus_fires.drop_duplicates()
aus_fires.info()

/home/due/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0,1,2,3,4,6,10,11,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(12131471, 15)

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,type
0,-32.756947,116.403992,335.46,0.8,0.78,2012-01-20,710,N,VIIRS,n,1,290.91,5.44,D,0
1,-19.087038,138.302383,306.25,0.67,0.74,2012-01-20,1503,N,VIIRS,n,1,290.15,2.23,N,0
2,-19.767071,147.392746,304.02,0.54,0.42,2012-01-20,1503,N,VIIRS,n,1,290.69,1.15,N,0
3,-19.772953,147.380814,306.19,0.54,0.42,2012-01-20,1503,N,VIIRS,n,1,290.69,1.13,N,0
4,-19.145247,139.365112,320.17,0.58,0.69,2012-01-20,1503,N,VIIRS,n,1,293.1,4.41,N,0


,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,type
0,-32.756947,116.403992,335.46,0.8,0.78,2012-01-20,710,N,VIIRS,n,1,290.91,5.44,D,0
1,-19.087038,138.302383,306.25,0.67,0.74,2012-01-20,1503,N,VIIRS,n,1,290.15,2.23,N,0
2,-19.767071,147.392746,304.02,0.54,0.42,2012-01-20,1503,N,VIIRS,n,1,290.69,1.15,N,0
3,-19.772953,147.380814,306.19,0.54,0.42,2012-01-20,1503,N,VIIRS,n,1,290.69,1.13,N,0
4,-19.145247,139.365112,320.17,0.58,0.69,2012-01-20,1503,N,VIIRS,n,1,293.1,4.41,N,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12131466,-20.612724,116.77346,310.05,0.44,0.63,2021-12-31,1806,N,VIIRS,n,1,295.76,2.19,N,2
12131467,-21.697954,115.002548,305.72,0.35,0.57,2021-12-31,1806,N,VIIRS,n,1,294.84,0.77,N,2
12131468,-20.597576,116.783463,331.56,0.44,0.63,2021-12-31,1806,N,VIIRS,n,1,295.79,2.24,N,2
12131469,-20.821804,115.386932,305.26,0.36,0.58,2021-12-31,1806,N,VIIRS,n,1,294.65,0.43,N,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12131471 entries, 0 to 12131470
Data columns (total 15 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   latitude    object
 1   longitude   object
 2   bright_ti4  object
 3   scan        object
 4   track       object
 5   acq_date    object
 6   acq_time    object
 7   satellite   object
 8   instrument  object
 9   confidence  object
 10  version     object
 11  bright_ti5  object
 12  frp         object
 13  daynight    object
 14  type        object
dtypes: object(15)
memory usage: 1.4+ GB


In [4]:
aus_fires['latitude'] = pd.to_numeric(aus_fires['latitude'], errors='coerce')
aus_fires['longitude'] = pd.to_numeric(aus_fires['longitude'], errors='coerce')
aus_fires['bright_ti4'] = pd.to_numeric(aus_fires['bright_ti4'], errors='coerce')
aus_fires['acq_date'] = pd.to_datetime(aus_fires['acq_date'], errors='coerce')
aus_fires['scan'] = pd.to_numeric(aus_fires['scan'], errors='coerce')
aus_fires['track'] = pd.to_numeric(aus_fires['track'], errors='coerce')
aus_fires['bright_ti5'] = pd.to_numeric(aus_fires['bright_ti5'], errors='coerce')
aus_fires['frp'] = pd.to_numeric(aus_fires['frp'], errors='coerce')


In [5]:
aus_fires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12131471 entries, 0 to 12131470
Data columns (total 15 columns):
 #   Column      Dtype         
---  ------      -----         
 0   latitude    float64       
 1   longitude   float64       
 2   bright_ti4  float64       
 3   scan        float64       
 4   track       float64       
 5   acq_date    datetime64[ns]
 6   acq_time    object        
 7   satellite   object        
 8   instrument  object        
 9   confidence  object        
 10  version     object        
 11  bright_ti5  float64       
 12  frp         float64       
 13  daynight    object        
 14  type        object        
dtypes: datetime64[ns](1), float64(7), object(7)
memory usage: 1.4+ GB


In [6]:
aus_fires.rename({'acq_date': 'time'}, axis=1, inplace=True)
aus_fires['est_fire_area'] = aus_fires['scan'] * aus_fires['track']
aus_fires['est_brightness'] = (aus_fires['bright_ti4'] + aus_fires['bright_ti5'])/2
aus_fires.latitude = aus_fires.latitude.round(1)
aus_fires.longitude = aus_fires.longitude.round(1)
aus_fires = aus_fires[aus_fires.type==0]

aus_fires.shape
aus_fires.head()

(11350427, 17)

,latitude,longitude,bright_ti4,scan,track,time,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,type,est_fire_area,est_brightness
0,-32.8,116.4,335.46,0.80,0.78,2012-01-20,710,N,VIIRS,n,1,290.91,5.44,D,0,0.6240,313.185
1,-19.1,138.3,306.25,0.67,0.74,2012-01-20,1503,N,VIIRS,n,1,290.15,2.23,N,0,0.4958,298.200
2,-19.8,147.4,304.02,0.54,0.42,2012-01-20,1503,N,VIIRS,n,1,290.69,1.15,N,0,0.2268,297.355
3,-19.8,147.4,306.19,0.54,0.42,2012-01-20,1503,N,VIIRS,n,1,290.69,1.13,N,0,0.2268,298.440
4,-19.1,139.4,320.17,0.58,0.69,2012-01-20,1503,N,VIIRS,n,1,293.10,4.41,N,0,0.4002,306.635


In [7]:
fires = aus_fires[['latitude', 'longitude', 'time','confidence',
                           'est_fire_area','est_brightness','frp']].copy()
## Add fire_count column
count = fires.groupby(['latitude', 'longitude', 'time']).size().reset_index().rename(columns={0:'fire_count'})
fire_copy = fires.merge(count,how='outer', on=['latitude', 'longitude', 'time'])

fire_copy

,latitude,longitude,time,confidence,est_fire_area,est_brightness,frp,fire_count
0,-32.8,116.4,2012-01-20,n,0.6240,313.185,5.44,2
1,-32.8,116.4,2012-01-20,n,0.1978,300.785,1.76,2
2,-19.1,138.3,2012-01-20,n,0.4958,298.200,2.23,5
3,-19.1,138.3,2012-01-20,n,0.3072,300.670,1.74,5
4,-19.1,138.3,2012-01-20,n,0.3072,310.700,4.11,5
...,...,...,...,...,...,...,...,...
11350422,-29.0,127.8,2021-12-31,n,0.1404,310.270,1.52,5
11350423,-29.0,127.8,2021-12-31,n,0.1404,298.305,1.52,5
11350424,-29.0,127.8,2021-12-31,n,0.1404,297.760,2.18,5
11350425,-29.0,127.8,2021-12-31,n,0.1404,320.180,2.18,5


In [8]:
fire_copy.confidence = fire_copy.confidence.replace({'l': 20, 'n': 60, 'h': 100})

In [9]:
fire_copy2 = fire_copy.groupby(['latitude', 'longitude', 'time'])[['fire_count','confidence','est_brightness']].mean().reset_index()
fire_copy2

,latitude,longitude,time,fire_count,confidence,est_brightness
0,-43.6,146.8,2012-03-28,4.0,60.0,298.308750
1,-43.6,146.8,2012-03-29,2.0,60.0,309.850000
2,-43.6,146.8,2014-04-01,2.0,60.0,314.877500
3,-43.6,146.8,2014-04-02,2.0,60.0,311.387500
4,-43.6,146.9,2014-04-01,1.0,60.0,321.845000
...,...,...,...,...,...,...
1323307,-9.4,142.7,2017-08-19,3.0,60.0,306.601667
1323308,-9.4,142.7,2018-06-02,1.0,60.0,315.580000
1323309,-9.4,142.7,2018-08-20,4.0,70.0,320.708750
1323310,-9.4,142.7,2018-09-05,2.0,60.0,307.075000


In [10]:
fire_copy3 = fire_copy.groupby(['latitude', 'longitude', 'time'])[['frp','est_fire_area']].sum().reset_index()
fire_copy3

,latitude,longitude,time,frp,est_fire_area
0,-43.6,146.8,2012-03-28,8.93,0.9812
1,-43.6,146.8,2012-03-29,8.46,0.3588
2,-43.6,146.8,2014-04-01,453.58,0.4998
3,-43.6,146.8,2014-04-02,9.29,0.3840
4,-43.6,146.9,2014-04-01,19.76,0.2499
...,...,...,...,...,...
1323307,-9.4,142.7,2017-08-19,7.82,0.8184
1323308,-9.4,142.7,2018-06-02,4.15,0.1404
1323309,-9.4,142.7,2018-08-20,64.54,0.7040
1323310,-9.4,142.7,2018-09-05,5.20,0.3780


In [11]:
fire_copy

,latitude,longitude,time,confidence,est_fire_area,est_brightness,frp,fire_count
0,-32.8,116.4,2012-01-20,60,0.6240,313.185,5.44,2
1,-32.8,116.4,2012-01-20,60,0.1978,300.785,1.76,2
2,-19.1,138.3,2012-01-20,60,0.4958,298.200,2.23,5
3,-19.1,138.3,2012-01-20,60,0.3072,300.670,1.74,5
4,-19.1,138.3,2012-01-20,60,0.3072,310.700,4.11,5
...,...,...,...,...,...,...,...,...
11350422,-29.0,127.8,2021-12-31,60,0.1404,310.270,1.52,5
11350423,-29.0,127.8,2021-12-31,60,0.1404,298.305,1.52,5
11350424,-29.0,127.8,2021-12-31,60,0.1404,297.760,2.18,5
11350425,-29.0,127.8,2021-12-31,60,0.1404,320.180,2.18,5


In [12]:
fire_copy = fire_copy2.merge(fire_copy3,how='outer', on=['latitude', 'longitude', 'time'])


In [13]:
fire_copy

,latitude,longitude,time,fire_count,confidence,est_brightness,frp,est_fire_area
0,-43.6,146.8,2012-03-28,4.0,60.0,298.308750,8.93,0.9812
1,-43.6,146.8,2012-03-29,2.0,60.0,309.850000,8.46,0.3588
2,-43.6,146.8,2014-04-01,2.0,60.0,314.877500,453.58,0.4998
3,-43.6,146.8,2014-04-02,2.0,60.0,311.387500,9.29,0.3840
4,-43.6,146.9,2014-04-01,1.0,60.0,321.845000,19.76,0.2499
...,...,...,...,...,...,...,...,...
1323307,-9.4,142.7,2017-08-19,3.0,60.0,306.601667,7.82,0.8184
1323308,-9.4,142.7,2018-06-02,1.0,60.0,315.580000,4.15,0.1404
1323309,-9.4,142.7,2018-08-20,4.0,70.0,320.708750,64.54,0.7040
1323310,-9.4,142.7,2018-09-05,2.0,60.0,307.075000,5.20,0.3780


In [14]:
## Add location count
count = fire_copy.groupby(['latitude', 'longitude']).size().reset_index().rename(columns={0:'loc_count'})
fire_copy = fire_copy.merge(count,how='outer', on=['latitude', 'longitude'])

fire_copy

,latitude,longitude,time,fire_count,confidence,est_brightness,frp,est_fire_area,loc_count
0,-43.6,146.8,2012-03-28,4.0,60.0,298.308750,8.93,0.9812,4
1,-43.6,146.8,2012-03-29,2.0,60.0,309.850000,8.46,0.3588,4
2,-43.6,146.8,2014-04-01,2.0,60.0,314.877500,453.58,0.4998,4
3,-43.6,146.8,2014-04-02,2.0,60.0,311.387500,9.29,0.3840,4
4,-43.6,146.9,2014-04-01,1.0,60.0,321.845000,19.76,0.2499,1
...,...,...,...,...,...,...,...,...,...
1323307,-9.4,142.7,2017-08-19,3.0,60.0,306.601667,7.82,0.8184,13
1323308,-9.4,142.7,2018-06-02,1.0,60.0,315.580000,4.15,0.1404,13
1323309,-9.4,142.7,2018-08-20,4.0,70.0,320.708750,64.54,0.7040,13
1323310,-9.4,142.7,2018-09-05,2.0,60.0,307.075000,5.20,0.3780,13


In [15]:
fire_copy.est_fire_area = fire_copy.est_fire_area.round(1)
fire_copy.est_brightness = fire_copy.est_brightness.round(1)
fire_copy.confidence = fire_copy.confidence.round().astype(int)
fire_copy.frp = fire_copy.frp.round(1)
fire_copy.fire_count = fire_copy.fire_count.round().astype(int)

fire_copy.head(20)

,latitude,longitude,time,fire_count,confidence,est_brightness,frp,est_fire_area,loc_count
0,-43.6,146.8,2012-03-28,4,60,298.3,8.9,1.0,4
1,-43.6,146.8,2012-03-29,2,60,309.8,8.5,0.4,4
2,-43.6,146.8,2014-04-01,2,60,314.9,453.6,0.5,4
3,-43.6,146.8,2014-04-02,2,60,311.4,9.3,0.4,4
4,-43.6,146.9,2014-04-01,1,60,321.8,19.8,0.2,1
5,-43.5,146.2,2018-03-23,10,60,320.1,219.5,1.5,1
6,-43.5,146.8,2014-02-07,15,60,298.0,50.0,3.6,4
7,-43.5,146.8,2014-02-08,10,64,318.4,139.0,1.6,4
8,-43.5,146.8,2020-03-26,1,60,278.9,0.5,0.2,4
9,-43.5,146.8,2020-06-10,1,60,306.1,4.2,0.2,4


In [16]:
confidence_norm = preprocessing.normalize([fire_copy['confidence']], norm='max')
frp_norm = preprocessing.normalize([fire_copy['frp']], norm='max')
area_norm = preprocessing.normalize([fire_copy['est_fire_area']], norm='max')
count_norm = preprocessing.normalize([fire_copy['fire_count']], norm='max')

In [17]:
fire_copy['ranking'] = confidence_norm[0]*0.4 + frp_norm[0]*0.2 + area_norm[0]*0.2 + count_norm[0]*0.2

In [18]:
fire_copy = fire_copy.sort_values(by=['ranking'], ascending=False)
fire_copy

,latitude,longitude,time,fire_count,confidence,est_brightness,frp,est_fire_area,loc_count,ranking
298644,-29.5,123.9,2019-02-16,315,79,331.6,358982.2,103.9,4,0.694892
233698,-31.4,123.7,2019-11-10,680,71,324.5,24134.6,207.1,11,0.667022
192588,-32.4,123.5,2020-01-02,474,86,335.9,11099.8,138.2,12,0.601851
363355,-27.6,144.3,2013-01-13,566,68,313.9,13568.5,185.4,2,0.599751
404639,-26.3,150.7,2013-10-08,551,66,311.2,10181.1,188.4,51,0.589020
...,...,...,...,...,...,...,...,...,...,...
122736,-34.1,138.9,2013-04-10,1,20,322.5,0.6,0.1,42,0.080346
131858,-33.9,135.3,2018-09-30,1,20,317.5,0.5,0.1,5,0.080346
207562,-32.0,118.4,2019-03-11,1,20,321.4,0.5,0.1,28,0.080346
557299,-22.2,134.5,2013-11-14,1,20,346.4,0.5,0.1,9,0.080346


In [22]:
fire_copy.sort_values(by=['time'], ascending=True)

,latitude,longitude,time,fire_count,confidence,est_brightness,frp,est_fire_area,loc_count,ranking
379966,-27.0,145.8,2012-01-20,8,60,301.8,12.0,1.7,4,0.243643
682796,-18.4,129.1,2012-01-20,1,60,294.8,1.6,0.2,9,0.240443
472202,-24.7,115.0,2012-01-20,3,60,300.2,11.3,1.3,4,0.242010
488248,-24.3,147.1,2012-01-20,4,60,300.8,6.8,0.8,22,0.241774
631525,-19.8,147.4,2012-01-20,2,60,297.9,2.3,0.5,31,0.240983
...,...,...,...,...,...,...,...,...,...,...
570528,-21.7,116.5,2021-12-31,1,20,319.1,2.8,0.2,15,0.080444
809203,-16.5,129.8,2021-12-31,12,60,309.4,39.6,1.9,24,0.244849
654055,-19.1,129.7,2021-12-31,1,60,326.8,11.9,0.2,5,0.240449
618581,-20.2,144.0,2021-12-31,10,60,301.5,22.4,3.8,14,0.246176


In [19]:
import sweetviz as sv
analyze_report = sv.analyze(fire_copy)
analyze_report.show_html(f'{fileName}.html', open_browser=False)


Report ../data/australia_viirs.html was generated.


In [20]:
fire_copy.to_csv(f"{fileName}_prepared.csv.gz", index=False, compression='gzip')
fire_copy.to_csv(f"{fileName}_prepared.csv", index=False)